<a href="https://colab.research.google.com/github/CesarAF10/Simulaci-n-I/blob/main/Invntarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import random
import math
# --- 1: DEFINICIÓN DE LOS DATOS DEL PROBLEMA ---

# 1.1 La Demanda Mensual: Cuánta gente podría pedir el producto.
demanda_probabilidades = {
    35: 0.010, 36: 0.015, 37: 0.020, 38: 0.020, 39: 0.022,
    40: 0.023, 41: 0.025, 42: 0.027, 43: 0.028, 44: 0.029,
    45: 0.035, 46: 0.045, 47: 0.060, 48: 0.065, 49: 0.070,
    50: 0.080, 51: 0.075, 52: 0.070, 53: 0.065, 54: 0.060,
    55: 0.050, 56: 0.040, 57: 0.030, 58: 0.016, 59: 0.015,
    60: 0.005
}

# 1.2 El Tiempo de Entrega: Cuánto tarda un nuevo pedido en llegar.
tiempo_entrega_probabilidades = {
    1: 0.30, # 1 mes de entrega es 30% probable
    2: 0.40, # 2 meses de entrega es 40% probable
    3: 0.30  # 3 meses de entrega es 30% probable
}

# 1.3 Factores Estacionales: La demanda cambia según el mes.
# En algunos meses se vende más (factor > 1), en otros menos (factor < 1).
factores_por_mes = {
    1: 1.20, 2: 1.00, 3: 0.90, 4: 0.80, 5: 0.80, 6: 0.70,
    7: 0.80, 8: 0.90, 9: 1.00, 10: 1.20, 11: 1.30, 12: 1.40
}

# 1.4 Los Costos:
costo_por_ordenar = 100  # Cuesta $100 cada vez que hacemos un pedido
costo_por_tener_inventario_anual = 20  # Cuesta $20 por unidad al año tenerla guardada
costo_por_faltante = 50  # Cuesta $50 por cada unidad que nos falta y no podemos vender

# --- 2: FUNCIONES PARA SIMULAR EVENTOS ALEATORIOS ---

def obtener_valor_aleatorio(distribucion):
    """
    Esta función simula un evento aleatorio basado en una tabla de probabilidades.
    Por ejemplo, si le pasamos las probabilidades de demanda, nos dará una demanda.
    """
    numero_aleatorio = random.random()
    probabilidad_acumulada = 0.0

    for valor, probabilidad in distribucion.items():
        probabilidad_acumulada += probabilidad
        if numero_aleatorio < probabilidad_acumulada:
            return valor # Cuando nuestro número aleatorio es menor que la suma, elegimos ese valor

    # Esto es para asegurar que siempre regrese un valor
    return list(distribucion.keys())[-1]

def simular_demanda_mensual(mes_actual):
    """
    Calcula la demanda para un mes específico, considerando la estación del año.
    """
    demanda_base = obtener_valor_aleatorio(demanda_probabilidades)
    factor_estacional = factores_por_mes[mes_actual]
    demanda_ajustada = round(demanda_base * factor_estacional) # Redondeamos porque no podemos pedir media unidad
    return int(demanda_ajustada) # Aseguramos que sea un número entero

def simular_tiempo_de_entrega():
    """
    Determina cuánto tiempo tardará un pedido en llegar.
    """
    return obtener_valor_aleatorio(tiempo_entrega_probabilidades)

# --- 3: LA SIMULACIÓN DEL INVENTARIO (EL CORAZÓN DEL PROGRAMA) ---

def simular_inventario_por_años(q_orden, R_reorden, inventario_inicial_al_principio, numero_de_años_a_simular=1, mostrar_detalle=False):
    """
    Simula el sistema de inventario para una cierta cantidad de años.
    q_orden: Cuántas unidades pedimos cada vez.
    R_reorden: Cuando nuestro inventario baja de este número, hacemos un pedido.
    inventario_inicial_al_principio: Con cuánto inventario empezamos el primer mes.
    numero_de_años_a_simular: Cuántos años queremos simular (ej. 30 años para un buen promedio).
    mostrar_detalle: Si es True, imprime todo el detalle mes a mes (útil para revisar).
    """

    inventario_actual = inventario_inicial_al_principio

    costo_total_ordenar = 0
    costo_total_por_inventario = 0
    costo_total_por_faltante = 0

    # Guardamos los pedidos que están "en camino" (cantidad, mes_de_llegada_absoluto)
    pedidos_en_camino = []

    if mostrar_detalle:
        print(f"\n--- INICIANDO SIMULACIÓN para Q={q_orden}, R={R_reorden}, Duración={numero_de_años_a_simular} Años ---")
        print(f"{'Año':<5}{'Mes':<5}{'Inv. Inicio':<15}{'Demanda':<12}{'Inv. Final':<13}{'Faltante':<10}{'Orden?':<7}{'Inv. Prom.':<15}{'Costo Inv. Mes':<20}")

    # Simulamos año por año, y dentro de cada año, mes por mes.
    for anio in range(1, numero_de_años_a_simular + 1):
        for mes_relativo_al_anio in range(1, 13):
            # Calculamos el número de mes "absoluto" para rastrear los pedidos
            mes_absoluto_global = (anio - 1) * 12 + mes_relativo_al_anio

            # 1. Al principio de cada mes, vemos si algún pedido en camino llegó
            pedidos_que_llegaron = 0
            nuevos_pedidos_en_camino = []
            for cantidad_pedido, mes_llegada_absoluto in pedidos_en_camino:
                if mes_llegada_absoluto == mes_absoluto_global:
                    inventario_actual += cantidad_pedido # Sumamos el pedido al inventario
                    pedidos_que_llegaron += cantidad_pedido
                else:
                    nuevos_pedidos_en_camino.append((cantidad_pedido, mes_llegada_absoluto))
            pedidos_en_camino = nuevos_pedidos_en_camino # Actualizamos la lista de pedidos en camino

            inventario_al_inicio_del_mes = inventario_actual

            # 2. Simulamos la demanda para este mes y atendemos los pedidos
            demanda_simulada = simular_demanda_mensual(mes_relativo_al_anio)

            unidades_faltantes = 0
            if inventario_actual >= demanda_simulada:
                inventario_actual -= demanda_simulada # Si hay suficiente, restamos la demanda
            else:
                unidades_faltantes = demanda_simulada - inventario_actual # Si no, calculamos el faltante
                inventario_actual = 0 # El inventario se vacía

            costo_total_por_faltante += unidades_faltantes * costo_por_faltante # Sumamos el costo por faltantes

            # 3. Revisamos si necesitamos hacer un nuevo pedido
            se_ordeno = ""
            # Si el inventario actual es igual o menor al punto de reorden (R), hacemos un pedido.
            if inventario_actual <= R_reorden:
                tiempo_estimado_entrega = simular_tiempo_de_entrega()
                mes_de_llegada_estimado = mes_absoluto_global + tiempo_estimado_entrega
                pedidos_en_camino.append((q_orden, mes_de_llegada_estimado)) # Añadimos el nuevo pedido a la lista de "en camino"
                costo_total_ordenar += costo_por_ordenar # Sumamos el costo de hacer la orden
                se_ordeno = "Si"

            # 4. Calculamos el inventario final y los costos de mantener inventario
            inventario_al_final_del_mes = inventario_actual

            # Calculamos el inventario promedio POSITIVO
            inventario_promedio_mensual_positivo = (inventario_al_inicio_del_mes + inventario_al_final_del_mes) / 2
            if inventario_promedio_mensual_positivo < 0: # Si el inventario "promedio" es negativo (por faltante), el costo es 0
                inventario_promedio_mensual_positivo = 0

            costo_mantener_inventario_este_mes = inventario_promedio_mensual_positivo * (costo_por_tener_inventario_anual / 12) # Costo mensual
            costo_total_por_inventario += costo_mantener_inventario_este_mes

            if mostrar_detalle:
                print(f"{anio:<5}{mes_relativo_al_anio:<5}{inventario_al_inicio_del_mes:<15}{demanda_simulada:<12}{inventario_al_final_del_mes:<13}{unidades_faltantes:<10}{se_ordeno:<7}{inventario_promedio_mensual_positivo:<15.2f}{costo_mantener_inventario_este_mes:<20.2f}")

    # Calculamos el costo total promedio anual al final de toda la simulación
    costo_promedio_anual_final = (costo_total_ordenar + costo_total_por_inventario + costo_total_por_faltante) / numero_de_años_a_simular

    if mostrar_detalle:
        print("\n--- RESUMEN DE COSTOS TOTALES AL FINAL DE LA SIMULACIÓN ---")
        print(f"Costo Total por Ordenar ({numero_de_años_a_simular} años): ${costo_total_ordenar:.2f}")
        print(f"Costo Total por Tener Inventario ({numero_de_años_a_simular} años): ${costo_total_por_inventario:.2f}")
        print(f"Costo Total por Faltantes ({numero_de_años_a_simular} años): ${costo_total_por_faltante:.2f}")
        print(f"Costo Promedio Anual (promedio de los {numero_de_años_a_simular} años): ${costo_promedio_anual_final:.2f}")

    return costo_promedio_anual_final

# --- 4. EL ALGORITMO DE OPTIMIZACIÓN (HOOKE Y JEEVES) ---

def algoritmo_hooke_jeeves(funcion_a_minimizar, punto_de_inicio, tamaño_de_paso_inicial, tolerancia_convergencia, max_iteraciones=1000, años_para_simular_en_optimizacion=30, inventario_inicial_para_opt=150, mostrar_progreso_opt=False):
    """
    Este es el algoritmo de Hooke y Jeeves para encontrar el mínimo de una función.

    funcion_a_minimizar: La función que queremos que sea lo más pequeña posible (nuestra simulación de inventario).
                         Debe tomar una lista [q, R], el inventario inicial y el número de años,
                         y devolver un costo.
    punto_de_inicio: Donde empezamos a buscar (ej. [200, 100] para Q y R).
    tamaño_de_paso_inicial: Qué tan grandes son los "pasos" al principio.
    tolerancia_convergencia: Si los pasos se vuelven muy pequeños, paramos la búsqueda.
    max_iteraciones: Un límite para que no busque infinitamente.
    años_para_simular_en_optimizacion: Cuántos años simula en cada evaluación para tener un costo estable.
    inventario_inicial_para_opt: Inventario inicial para cada simulación durante la optimización.
    mostrar_progreso_opt: Si es True, imprime el proceso de búsqueda.
    """

    def funcion_objetivo_adaptada(valores_q_r):
        q_valor = max(1, int(round(valores_q_r[0]))) # Q debe ser un entero positivo
        r_valor = max(0, int(round(valores_q_r[1]))) # R debe ser un entero no negativo

        return funcion_a_minimizar(q_valor, r_valor, inventario_inicial_para_opt, numero_de_años_a_simular=años_para_simular_en_optimizacion, mostrar_detalle=False)

    punto_base_actual = list(punto_de_inicio) # Nuestro mejor punto actual
    punto_patron = list(punto_de_inicio)       # Un punto para dar "saltos" más grandes

    numero_de_variables = len(punto_de_inicio)

    # Calculamos el costo inicial en nuestro punto de partida
    mejor_costo_encontrado = funcion_objetivo_adaptada(punto_base_actual)

    iteracion = 0
    paso_actual = tamaño_de_paso_inicial # El tamaño de paso que usaremos

    # El bucle principal del algoritmo: seguimos buscando mientras el paso sea grande y no hayamos llegado al límite de iteraciones
    while paso_actual > tolerancia_convergencia and iteracion < max_iteraciones:
        iteracion += 1

        # --- ETAPA 1: BÚSQUEDA EXPLORATORIA ---
        # Intentamos movernos un poco en cada dirección para ver si mejora.

        candidato_nuevo_punto_base = list(punto_base_actual)
        hubo_mejora_en_exploracion = False

        for i in range(numero_de_variables): # Para Q y luego para R
            valor_original = candidato_nuevo_punto_base[i]

            # 1. Probar moviendo en la dirección positiva (sumando el paso)
            candidato_nuevo_punto_base[i] = valor_original + paso_actual
            costo_con_mas_paso = funcion_objetivo_adaptada(candidato_nuevo_punto_base)

            if costo_con_mas_paso < mejor_costo_encontrado:
                mejor_costo_encontrado = costo_con_mas_paso
                hubo_mejora_en_exploracion = True
            else:
                # 2. Si no mejora, probar moviendo en la dirección negativa (restando el paso)
                candidato_nuevo_punto_base[i] = valor_original - paso_actual
                costo_con_menos_paso = funcion_objetivo_adaptada(candidato_nuevo_punto_base)

                if costo_con_menos_paso < mejor_costo_encontrado:
                    mejor_costo_encontrado = costo_con_menos_paso
                    hubo_mejora_en_exploracion = True
                else:
                    # 3. Si ninguna dirección mejora, volvemos al valor original de esta variable
                    candidato_nuevo_punto_base[i] = valor_original

        # --- ETAPA 2: MOVIMIENTO PATRÓN O REDUCCIÓN DE PASO ---

        if hubo_mejora_en_exploracion:
            # Si encontramos algo mejor en la exploración, movemos nuestro punto base a ese nuevo y mejor punto.
            punto_base_actual = list(candidato_nuevo_punto_base)

            # Luego, intentamos dar un "salto" más grande en esa dirección (el punto patrón)
            # Este salto es el doble de la distancia que nos movimos desde el patrón anterior hacia el nuevo punto base.
            punto_patron = [2 * punto_base_actual[i] - punto_patron[i] for i in range(numero_de_variables)]

            # Evaluamos el costo en este nuevo punto patrón
            costo_con_patron = funcion_objetivo_adaptada(punto_patron)
            if costo_con_patron < mejor_costo_encontrado:
                mejor_costo_encontrado = costo_con_patron
            else:
                # Si el punto patrón no mejoró, el nuevo punto patrón es simplemente el punto base actual.
                punto_patron = list(punto_base_actual)

        else:
            # Si no hubo mejora en la exploración, significa que estamos cerca de un mínimo.
            # Hacemos los pasos más pequeños para buscar con más detalle.
            paso_actual /= 2.0
            # Y reiniciamos el punto patrón a nuestro punto base actual.
            punto_patron = list(punto_base_actual)

        if mostrar_progreso_opt:
            q_actual_redondeado = int(round(punto_base_actual[0]))
            r_actual_redondeado = int(round(punto_base_actual[1]))
            print(f"Iteración {iteracion}: Paso={paso_actual:.2f}, Q={q_actual_redondeado}, R={r_actual_redondeado}, Costo Actual=${mejor_costo_encontrado:.2f}")

    return punto_base_actual, mejor_costo_encontrado # Devolvemos los mejores Q y R y su costo

# --- 5: EJECUCIÓN DEL PROGRAMA PRINCIPAL ---


if __name__ == "__main__":
    print("--- EMPEZANDO EL PROGRAMA DE OPTIMIZACIÓN DE INVENTARIO ---")

    # --- Primero, probamos la simulación una vez con valores de ejemplo ---

    print("\n--- SIMULACIÓN DE UN AÑO CON Q=200, R=100 (valores del problema manual) ---")
    random.seed(42) # Usamos una semilla fija para que esta parte sea reproducible
    costo_simulado_ejemplo = simular_inventario_por_años(q_orden=200, R_reorden=100, inventario_inicial_al_principio=150, numero_de_años_a_simular=1, mostrar_detalle=True)
    print(f"\nCosto Promedio Anual (Q=200, R=100, 1 año de simulación): ${costo_simulado_ejemplo:.2f}")
    print("-" * 70)

    # --- Ahora, preparamos la búsqueda del Q y R óptimos usando Hooke y Jeeves ---
    print("\n--- INICIANDO LA BÚSQUEDA DEL Q Y R ÓPTIMOS CON HOOKE Y JEEVES ---")

    # Definimos los parámetros para el algoritmo de Hooke y Jeeves:
    punto_inicial_para_optimizacion = [200.0, 100.0] # Empezamos la búsqueda desde Q=200, R=100
    paso_inicial_optimizacion = 50.0          # El tamaño de los primeros "pasos"
    tolerancia_optimizacion = 1.0             # Cuando el paso sea menor a 1.0, consideramos que encontramos un buen punto
    años_para_cada_simulacion = 30            # Cada vez que Hooke y Jeeves pruebe un Q y R, simulará 30 años
    maximo_de_iteraciones_hooke_jeeves = 200  # Máximo de intentos para Hooke y Jeeves
    mostrar_proceso_optimizacion = True       # Queremos ver cómo avanza la búsqueda

    # Llamamos al algoritmo de Hooke y Jeeves
    valores_optimos_encontrados, costo_minimo_estimado = algoritmo_hooke_jeeves(
        funcion_a_minimizar=simular_inventario_por_años,
        punto_de_inicio=punto_inicial_para_optimizacion,
        tamaño_de_paso_inicial=paso_inicial_optimizacion,
        tolerancia_convergencia=tolerancia_optimizacion,
        max_iteraciones=maximo_de_iteraciones_hooke_jeeves,
        años_para_simular_en_optimizacion=años_para_cada_simulacion,
        inventario_inicial_para_opt=150, # Usamos el inventario inicial dado en el problema
        mostrar_progreso_opt=mostrar_proceso_optimizacion
    )

    # Redondeamos los resultados finales a números enteros
    q_optimo_final = int(round(valores_optimos_encontrados[0]))
    r_optimo_final = int(round(valores_optimos_encontrados[1]))

    print("\n--- RESULTADOS FINALES DE LA OPTIMIZACIÓN ---")
    print(f"La Cantidad de Pedido Óptima (Q) es: {q_optimo_final}")
    print(f"El Nivel de Reorden Óptimo (R) es: {r_optimo_final}")
    print(f"El Costo Total Anual Estimado Mínimo es: ${costo_minimo_estimado:.2f}")
    print("\n--- VERIFICANDO EL COSTO CON LOS VALORES ÓPTIMOS ---")
    random.seed(random.randint(0, 999999))
    costo_verificacion_final = simular_inventario_por_años(
        q_orden=q_optimo_final,
        R_reorden=r_optimo_final,
        inventario_inicial_al_principio=150,
        numero_de_años_a_simular=años_para_cada_simulacion * 2, # Simulamos el doble de años para mayor precisión
        mostrar_detalle=False
    )
    print(f"Costo verificado con Q={q_optimo_final}, R={r_optimo_final} en {años_para_cada_simulacion * 2} años: ${costo_verificacion_final:.2f}")

--- EMPEZANDO EL PROGRAMA DE OPTIMIZACIÓN DE INVENTARIO ---

--- SIMULACIÓN DE UN AÑO CON Q=200, R=100 (valores del problema manual) ---

--- INICIANDO SIMULACIÓN para Q=200, R=100, Duración=1 Años ---
Año  Mes  Inv. Inicio    Demanda     Inv. Final   Faltante  Orden? Inv. Prom.     Costo Inv. Mes      
1    1    150            61          89           0         Si     119.50         199.17              
1    2    289            46          243          0                266.00         443.33              
1    3    243            40          203          0                223.00         371.67              
1    4    203            42          161          0                182.00         303.33              
1    5    161            42          119          0                140.00         233.33              
1    6    119            38          81           0         Si     100.00         166.67              
1    7    281            38          243          0                262.00    